## Can we predict in how many months an employee would leave?

### General Information
-Data pulled from HRIS

-Data dates back to 2008

-Possible missing data due to user input/lack of information

### Challenges
-Small data set (1,815 rows)

-Small number of variables

-Little to no correlated variables

### Load up the termination data file

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
plt.style.use('fivethirtyeight')

%matplotlib inline

data = pd.read_csv('test_data.csv') #Import data set
data = data.drop('Employee ID', axis = 1)

In [2]:
data.columns  #Check current columns before making dummy columns

Index(['Business Title', 'Job Profile', 'Gender',
       'Length of Service in Months', 'Is Manager', 'Employee Type',
       'Cost Center', 'Company', 'Location', 'Region', 'Termination Category',
       'Termination Reason', 'Job Family', 'Primary Work Address - Country',
       'Work Address - City', 'Total Base Pay - Amount',
       'Total Base Pay - Currency', 'Compa-Ratio',
       'Last Base Pay Increase - Date', 'Last Base Pay Increase - Percent'],
      dtype='object')

In [3]:
dummies = pd.get_dummies(data, columns=['Termination Category', 'Termination Reason','Gender','Is Manager','Job Profile','Job Family','Company','Location','Job Family','Work Address - City','Business Title','Region','Primary Work Address - Country'])
data = dummies ##Create dummy columns for string values

In [4]:
data.shape #Only 1815 rows

(1815, 1720)

In [5]:
data.head()

,Length of Service in Months,Employee Type,Cost Center,Total Base Pay - Amount,Total Base Pay - Currency,Compa-Ratio,Last Base Pay Increase - Date,Last Base Pay Increase - Percent,Termination Category_Forced Attrition,Termination Category_Voluntary Attrition,...,Primary Work Address - Country_Netherlands,Primary Work Address - Country_Norway,Primary Work Address - Country_Poland,Primary Work Address - Country_Russian Federation,Primary Work Address - Country_Singapore,Primary Work Address - Country_Spain,Primary Work Address - Country_Sweden,Primary Work Address - Country_Turkey,Primary Work Address - Country_United Kingdom,Primary Work Address - Country_United States of America
0,18,Permanent,1060 Label Relations - DO NOT USE,"90,000.00",USD,1.055,9/26/11,0.0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,10,Permanent,3007 Business Operations - DO NOT USE,"140,000.00",USD,1.496,6/4/12,0.0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,11,Permanent,4010 Product Development & Design,"125,000.00",USD,0.906,5/15/12,0.0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1,Permanent,1050 Human Resources,"117,000.00",USD,1.146,2/25/13,0.0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,32,Permanent,3010 Ad Sales Staff - DO NOT USE,"1,260,000.00",SEK,1.277,12/1/12,0.0,1,0,...,0,0,0,0,0,0,1,0,0,0


## Let's try playing around with different machine learning techniques

### Decision Tree Attempt

In [6]:
from sklearn.tree import DecisionTreeRegressor

target_col = 'Length of Service in Months'

X = data.select_dtypes('number').dropna(axis='columns').drop(target_col, axis='columns')
y = data.loc[:,target_col]

dtr = DecisionTreeRegressor(max_depth=3)
dtr.fit(X, y)
dtr.score(X, y)

0.30417212999643517

Horrible score for Decision Tree, lets pull an image of the tree below:

In [7]:
images_dir = '../Final_Project/assets/images'
dotfile_path = f'{images_dir}/term_tree.dot'
png_path = f'{images_dir}/term_tree.png'

In [8]:
from sklearn.tree import export_graphviz

export_graphviz(dtr, out_file=dotfile_path, feature_names=X.columns)

In [9]:
!dot -Tpng {dotfile_path} -o {png_path}

In [10]:
tree_image = plt.imread(png_path)
fig, ax = plt.subplots(figsize=(200, 200))
ax.imshow(tree_image);

### Logical Regression Attempt w/ Train Test Split

In [11]:
X = data.select_dtypes('number').dropna(axis='columns').drop(target_col, axis='columns')
y = data.loc[:,target_col]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Import a model class
from sklearn.linear_model import LogisticRegression

# Create a model from that class
lr = LogisticRegression()

# Ask the model to learn a function that predicts `y` from `X`
lr.fit(X_train, y_train)

/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
print('Training score:',lr.score(X_train, y_train))
print('Testing score:', lr.score(X_test, y_test))

Training score: 0.7942689199118296
Testing score: 0.05066079295154185


Model is overfitting. 

In [15]:
list(zip(lr.predict(X_test), y_test))

[(5, 6),
 (26, 47),
 (31, 8),
 (13, 59),
 (48, 28),
 (8, 79),
 (26, 85),
 (21, 13),
 (47, 29),
 (6, 1),
 (42, 27),
 (13, 69),
 (60, 45),
 (15, 2),
 (6, 5),
 (12, 59),
 (16, 34),
 (12, 67),
 (12, 0),
 (53, 56),
 (32, 37),
 (21, 85),
 (2, 14),
 (53, 73),
 (16, 3),
 (47, 45),
 (50, 36),
 (10, 38),
 (35, 23),
 (18, 13),
 (34, 55),
 (60, 46),
 (23, 113),
 (19, 31),
 (31, 32),
 (34, 25),
 (7, 5),
 (4, 80),
 (21, 12),
 (16, 5),
 (6, 2),
 (12, 12),
 (13, 18),
 (23, 34),
 (64, 24),
 (10, 12),
 (24, 53),
 (4, 23),
 (14, 15),
 (17, 25),
 (1, 8),
 (66, 13),
 (0, 17),
 (8, 2),
 (24, 33),
 (20, 4),
 (10, 49),
 (70, 12),
 (5, 61),
 (18, 15),
 (61, 57),
 (53, 62),
 (37, 101),
 (18, 40),
 (20, 19),
 (57, 67),
 (10, 21),
 (24, 9),
 (32, 79),
 (12, 11),
 (0, 0),
 (5, 10),
 (40, 78),
 (15, 10),
 (28, 22),
 (2, 50),
 (3, 8),
 (83, 64),
 (0, 60),
 (11, 89),
 (8, 6),
 (5, 3),
 (30, 21),
 (21, 15),
 (79, 66),
 (58, 21),
 (10, 7),
 (14, 37),
 (9, 0),
 (10, 37),
 (20, 31),
 (5, 4),
 (49, 18),
 (53, 24),
 (23, 2

### Random Forest Regressor Attempt w/ Train Test Split

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rfr = RandomForestRegressor(n_estimators=100)

rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
print('Training score:', rfr.score(X_train, y_train))
print('Testing score:', rfr.score(X_test, y_test))

Training score: 0.9008169597024516
Testing score: 0.2038857928614668


Training score is high, but testing score has too much variance against training score.

In [18]:
list(zip(rfr.predict(X_test), y_test))

[(5.1, 6),
 (4.84, 47),
 (10.89, 8),
 (35.2, 59),
 (38.4, 28),
 (40.46, 79),
 (40.31, 85),
 (39.34, 13),
 (30.93, 29),
 (5.59, 1),
 (48.71, 27),
 (22.88, 69),
 (51.18, 45),
 (10.15, 2),
 (5.42, 5),
 (47.76, 59),
 (44.41, 34),
 (22.31, 67),
 (3.27, 0),
 (47.2, 56),
 (32.89, 37),
 (36.46, 85),
 (36.03, 14),
 (69.24, 73),
 (12.89, 3),
 (30.59, 45),
 (41.03, 36),
 (11.06, 38),
 (32.77, 23),
 (11.85, 13),
 (71.01, 55),
 (62.17, 46),
 (32.09, 113),
 (49.05, 31),
 (30.15, 32),
 (34.47, 25),
 (12.34, 5),
 (42.85, 80),
 (23.93, 12),
 (4.18, 5),
 (6.57, 2),
 (11.95, 12),
 (24.12, 18),
 (34.42, 34),
 (40.07, 24),
 (16.81, 12),
 (51.0, 53),
 (32.88, 23),
 (43.8, 15),
 (70.64, 25),
 (24.76, 8),
 (2.87, 13),
 (30.84, 17),
 (5.79, 2),
 (39.68, 33),
 (6.28, 4),
 (27.16, 49),
 (39.63, 12),
 (40.2, 61),
 (8.95, 15),
 (39.4, 57),
 (51.87, 62),
 (43.0, 101),
 (46.0, 40),
 (21.54, 19),
 (46.0, 67),
 (20.65, 21),
 (34.02, 9),
 (39.68, 79),
 (9.08, 11),
 (19.62, 0),
 (10.76, 10),
 (48.45, 78),
 (9.34, 10),
 

### Random Forest Regressor w/ KFold

In [21]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=2)

fold = GridSearchCV(estimator=RandomForestRegressor(random_state=16),
                    param_grid={},
                    return_train_score=True,
                    cv=kf,
                   )
fold.fit(X, y)

/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/evankatz/anaconda3/lib/py

GridSearchCV(cv=KFold(n_splits=10, random_state=2, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=16, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [22]:
print('Training score:', rfr.score(X_train, y_train))
print('Testing score:', rfr.score(X_test, y_test))

Training score: 0.9008169597024516
Testing score: 0.2038857928614668


In [23]:
list(zip(fold.predict(X),y))

[(14.5, 18),
 (9.3, 10),
 (11.8, 11),
 (11.4, 1),
 (24.7, 32),
 (20.4, 15),
 (15.9, 21),
 (12.4, 13),
 (22.4, 36),
 (15.0, 20),
 (14.6, 17),
 (6.8, 5),
 (30.7, 25),
 (17.3, 13),
 (10.3, 12),
 (28.6, 38),
 (9.0, 5),
 (13.2, 7),
 (11.5, 14),
 (17.3, 30),
 (20.2, 28),
 (44.5, 42),
 (4.7, 2),
 (17.1, 13),
 (27.4, 23),
 (39.9, 43),
 (16.4, 24),
 (34.9, 29),
 (26.7, 23),
 (11.4, 8),
 (34.2, 31),
 (39.5, 34),
 (31.4, 53),
 (21.6, 26),
 (10.0, 10),
 (5.6, 6),
 (9.2, 9),
 (19.6, 22),
 (26.5, 25),
 (64.8, 79),
 (26.8, 36),
 (27.1, 28),
 (32.4, 27),
 (10.2, 2),
 (40.8, 41),
 (4.9, 5),
 (8.9, 4),
 (13.8, 11),
 (29.1, 27),
 (6.7, 7),
 (53.0, 55),
 (9.5, 8),
 (4.9, 5),
 (25.1, 22),
 (9.4, 10),
 (7.5, 5),
 (26.1, 21),
 (23.3, 27),
 (44.9, 50),
 (24.3, 41),
 (33.3, 32),
 (19.9, 27),
 (1.9, 1),
 (21.8, 26),
 (36.0, 41),
 (64.5, 85),
 (7.7, 10),
 (10.9, 11),
 (26.1, 27),
 (14.6, 14),
 (5.0, 3),
 (9.3, 6),
 (25.7, 26),
 (7.7, 3),
 (9.8, 7),
 (12.1, 13),
 (48.4, 57),
 (25.2, 25),
 (39.4, 6),
 (20.4, 29),


Training score accurate, not so much on testing, overfitting.

The fact that the model does well on the training set suggests that it is flexible enough. However, the fact that the model does substantially worse on the test set suggests that it is too flexible -- it is capturing random variations that happen to be present in the test set as well as the real underlying patterns that generalize beyond it.

Make model more simpler, but variables might be necessary -- add more data points/variables.

### Linear Regression Attempt w/ Train Test Split

In [24]:
X = data.select_dtypes('number').dropna(axis='columns').drop(target_col, axis='columns')
y = data.loc[:,target_col]

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [26]:
# Import a model class
from sklearn.linear_model import LinearRegression

# Create a model from that class
lr = LinearRegression()

# Ask the model to learn a function that predicts `y` from `X`
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [27]:
from sklearn.metrics import mean_squared_error

predictions = lr.predict(X_test)
np.sqrt(mean_squared_error(y_test, predictions))

print('Training score:', lr.score(X_train, y_train))
print('Testing score:', lr.score(X_test, y_test))

Training score: 0.8023781689762953
Testing score: -0.23789832612945383


Model is overfitting.

In [28]:
list(zip(lr.predict(X_test), y_test))

[(0.02037287266838561, 6),
 (23.6519135068336, 47),
 (67.71548841259553, 8),
 (21.242734533299892, 59),
 (29.819008203856363, 28),
 (27.661585026315738, 79),
 (44.38869222324702, 85),
 (2.690302421899503, 13),
 (31.25533304010411, 29),
 (-23.288987398166018, 1),
 (52.98970531252597, 27),
 (36.64516373546472, 69),
 (39.848153292195164, 45),
 (10.11352465106654, 2),
 (28.13201290384565, 5),
 (7.277439536887602, 59),
 (6.0707866169481655, 34),
 (45.88078026894581, 67),
 (8.579774525736141, 0),
 (38.046537731825666, 56),
 (39.07208816252508, 37),
 (-11.51357481217443, 85),
 (26.70477770938381, 14),
 (92.21854962942854, 73),
 (48.00087802419242, 3),
 (25.25999285545938, 45),
 (35.44536451110706, 36),
 (17.168558326543632, 38),
 (49.36502277062927, 23),
 (23.32696923890999, 13),
 (54.96899767460583, 55),
 (49.30974973230158, 46),
 (27.60739889407004, 113),
 (33.29546039730772, 31),
 (54.3895410801147, 32),
 (51.36579547673031, 25),
 (12.685498381907605, 5),
 (37.255136416612714, 80),
 (60.65

### Linear Regression Attempt w/ KFold

In [31]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)

lr_grid = GridSearchCV(estimator=lr, param_grid={}, cv=kf, return_train_score=True)
lr_grid.fit(X, y)
results = lr_grid.cv_results_
print('Train_score', results['mean_train_score'][0])
print('Test_score',results['mean_test_score'][0])

/Users/evankatz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Train_score 0.7817037121096002
Test_score -72791139620.78397


Model overfitting.

In [32]:
list(zip(lr_grid.predict(X),y))

[(18.00000001144355, 18),
 (9.999999990871665, 10),
 (11.000000000470706, 11),
 (0.9999999728154396, 1),
 (31.999999990420513, 32),
 (31.36470300559234, 15),
 (24.23034754347001, 21),
 (13.00000000162239, 13),
 (35.99999999415926, 36),
 (20.000000029417752, 20),
 (17.000000034204042, 17),
 (4.999999998485279, 5),
 (33.173591388369424, 25),
 (12.999999969140287, 13),
 (11.99999999921845, 12),
 (46.81675385634471, 38),
 (40.102784797441274, 5),
 (7.000000022704658, 7),
 (8.053467836931791, 14),
 (29.999999998241883, 30),
 (28.000000026643143, 28),
 (32.56763327592633, 42),
 (1.9999999867629796, 2),
 (22.508463843167938, 13),
 (23.00000003538515, 23),
 (36.465804958596756, 43),
 (17.2237524772995, 24),
 (28.999999999346585, 29),
 (22.99999997796762, 23),
 (8.000000002468756, 8),
 (30.999999993891436, 31),
 (33.99999999220801, 34),
 (53.00000000399346, 53),
 (20.770369882482843, 26),
 (9.999999968619598, 10),
 (5.999999986780352, 6),
 (9.000000012679248, 9),
 (21.999999994699017, 22),
 (37

##  Conclusion

Random Forest Regressor Attempt w/ Train Test Split (and w/ KFold) has the best performance when comparing against the other modeling techniques. The training set score was 90%, but the test set score was 21%.

This shows that the model is currently overfitting, and more engineering is required. At this point, I do not believe it's easy to make the model a better fit without additional steps.

### Next Steps

-Develop a more simpler model

-Add additional data (current data set is small in size 1,815 rows)

-Add additional correlated factors/variables (i.e. performance data)
